<small><font color=gray>Notebook authors: Margarita Prikhodko, Hunter, Seiyoung Caitlyn Lee</a> ©2023 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🍿Netflix</font>](https://www.kaggle.com/c/19jun23jh-netflix/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip install -q --upgrade imgaug==0.2.5 > log.txt  # avoids warning/conflict with scipy upgrade
!pip install -q --upgrade scipy >>log.txt  # upgrades 1.4 to 1.8

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 19jun23jh-netflix        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 19jun23jh-netflix
100% 45.0M/45.0M [00:02<00:00, 20.8MB/s]
Using competition: 19jun23jh-netflix
  teamId  teamName        submissionDate       score    
--------  --------------  -------------------  -------  
10602951  Group 9         2023-07-02 00:12:04  0.42990  
10602783  Group 7         2023-07-03 02:18:21  0.42490  
10606288  Group 3         2023-07-02 06:29:58  0.42270  
10609906  Group 10        2023-07-02 17:27:13  0.42150  
10607335  Group 4         2023-07-03 01:19:07  0.42020  
10580078  Group 12        2023-07-02 23:03:13  0.41960  
10602607  Team 13         2023-07-01 18:51:11  0.41900  
10580383  Group 5         2023-07-02 14:39:06  0.41870  
10607564  Team 14         2023-07-02 17:19:16  0.41800  
10615163  Group 2         2023-07-02 16:20:24  0.41690  
10603056  Group 6         2023-07-02 00:45:44  0.41670  
10592821  Group 1         2023-07-03 01:28:29  0.41660  
10601940  Team 8    

In [ ]:
%%time
%%capture
%reset -f
!cp metadata.json METADATA
!pip -q install scikit-surprise
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, plotly.express as px, surprise
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=1000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 641 ms, sys: 75 ms, total: 716 ms
Wall time: 14.1 s


The matrix below contains ratings that users (user id or `uid`) have given to the movies (movie ID or `mid`). It is 85% blank, but the original dataset was 99% sparse (with ~1 rating per user) [&#127910;](https://youtu.be/ImpV70uLxyw). Your task is not to fill the whole matrix, but to impute `NaN` values in `XY_Netflix.csv`, which is an equivalent representation of `XY_Netflix_matrix.csv`. This is a collaborative filtering problem, where observations and features have some implicit relation. Recall that typically we expect independent observations in a sample (unless we have a time series or video of image frames, etc.). Here our model needs to learn and leverage the dependencies from observed movie ratings. Clustering techniques might be more intuitive here. [Truncated SVD](https://scikit-learn.org/stable/modules/decomposition.html#lsa) is also popular. Supervised models can be used as well, if we can build features describing our users (which we would treat as observations). Note that we have no other information on users except what they have ranked (which might suggest the cluster they belong to, which might offer unobserved (latent) features). [&#127910;](https://youtu.be/8wLKuscyO9I) Since users relate to other users through the movies they ranked, this can also be presented as a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph).

In [ ]:
dfMtx = pd.read_csv('XY_Netflix_matrix.csv', low_memory=False); dfMtx

,1,2,3,4,5,6,7,8,9,10,...,371,372,373,374,375,376,377,378,379,380
0,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128698,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f = px.imshow(dfMtx.iloc[:2000,:].T, color_continuous_scale='RdBu_r', aspect='auto');  # movies by users (transposed for visual convenience)
f.update_layout(margin=dict(l=0,r=0,b=0,t=0))

In [ ]:
dfMtx.info()   # note the datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128699 entries, 0 to 128698
Columns: 380 entries, 1 to 380
dtypes: float64(380)
memory usage: 373.1 MB


The dataframe below is another representation of the matrix form above. It lists the `uid`$\times$`mid`locations of the values you need to predict (those with `NaN`) and all other non-zero values in the matrix above.

In [ ]:
df = pd.read_csv('XY_Netflix.csv', low_memory=False); df  # user ID, movie ID, rating. NaN=requires predictions

,uid,mid,rtg
0,1,1,NaN
1,2,2,NaN
...,...,...,...
7402070,49692,126,4.0
7402071,9783,37,1.0


In [ ]:
df.rtg.fillna(0).astype(int).value_counts().to_frame().T   # rating frequencies. 0 represents NaN

,4,3,5,2,1,0
rtg,2451087,2120585,1628122,842056,340222,20000


In [ ]:
dfMov = pd.read_csv('movies.csv'); dfMov.head(2)   # load movie attributes

,mid,Year,Name,voteAvg,voteCnt,popularity,duration,income,budget,releaseDate,...,origLang,spokenLang,ttl,origTtl,URL,kwds,desc,prodComp,prodCtry,tagline
0,1,2004,The Village,6.2,1071,27.49,108.0,256697520,60000000,7/30/04,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Village,The Village,NaN,"[{""id"": 1328, ""name"": ""secret""}, {""id"": 5774, ...",When a willful young man tries to venture beyo...,"[{""name"": ""Scott Rudin Productions"", ""id"": 258...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",There is no turning back
1,2,1997,Liar Liar,6.4,1424,26.72,86.0,181000000,45000000,3/21/97,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Liar Liar,Liar Liar,NaN,"[{""id"": 387, ""name"": ""california""}, {""id"": 102...",Fletcher Reede is a fast-talking attorney and ...,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",Trust me.


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Group 15 Contributers:**

Hunter Kalmut

Caitlyn Lee

Margarita Prikhodko


## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.

This project was tricky in that the baseline score seemed to be a fairly high score already. Many recommender systems at the enterprise level use truncated singular value decomposition (SVD) to attempt to improve recommendations [1].

That said, in terms of preprocessing, first the 20,000 unlabeled instances were filtered out of the main dataset and the users involved in these ratings were observed. This formed a mainly sparse data matrix with a bit over 18,000 unique users as rows and 380 movies as columns. To fill in the values which are mainly NaN, the team stuck with the imputing the mean value.

Next, this matrix was applied robust PCA which is a technique to separate the data matrix X into two new matrices X = L + S, where L is a low-rank matrix containing the de-noised observations and S is a sparse matrix containing the noise.

Then this new matrix L is applied Singular value decomposition over a series of iterations in the following manner: one iteration of SVD occurs to update the entire matrix L and then the rating elements which already existed and were not imputed are set back into the matrix. Thus, only artificial ratings were augmented which started as the column mean for that movie.

1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)

For the users for which a rating was to be predicted, a matrix was formed with users as rows and movies as columns. This matrix was employed based on ideas inspired by the following video[2].

Ultimately, the preprocessing and model selection are similar to that of the baseline.
Values in the user-movie matrix were imputed using the column mean. Admittedly, it is possible the overall average movie score may have been a better initializer than the column means for the users with unrated movies.

Robust PCA was employed from [3] in an effort to de-noise the data before applying singular value decomposition.

Singular value decomposition was employed in a similar manner to the baseline submission except with more iterations and more components referenced. These hyperparameters were tuned in a grid search with 5-fold cross-validation.
2. How do you evaluate the effectiveness of these elements?

In an iterative process, the team would make exactly one change to the preprocessing pipeline and then run an experiment via cross-validation on held-out test data. If evaluation scores for mean absolute error, root mean squared error, and classification accuracy for un-encountered test data improved on this test data, the team assumed the model was benefitting from the change. Though, it is  true even though test accuracy improved locally for some of the preprocessing steps, public leaderboard scores declined in certain circumstances, which seems unintuitive but may reflect the distribution of the unlabeled instances is not the same as the subsamples of users the team used for evaluation. Ultimately, though, lower error and higher classification accuracy were indicators for effectiveness.

3. What else have you tried that worked or didn't?

One element the team tried that did not appear to work was imputing the missing ratings in the user-movie matrix with the median value. Similarly, for certain movies, like Napoleon Dynamite, it was clear the ratings were polarizing, and so the team attempted to clip neutral ratings for these types of films. This may have injected too much bias into the end model, for classification accuracy scores decreased.

Also, the team tried to form a column MovieGenre based on the Genre bit flag columns in the Netflix dataset in an attempt to group movies together relative to the users. However, with this information, the team was not exactly sure how to incorporate it into the data model for the SVD.
The team tried suprise which is part of a python Scikit library that is suitable for recommendar systems analysis such as the netflix prize application. The team found connections between users and items. Matrix of connections.
The team experimented with multiple algorithms available in the surprise library, including Kmeans, slopeOne, and Matrix Factorization-based approaches like SVD and SVD++. Despite extensive hyperparameter tuning and analysis, the team was unable to increase our accuracy score, and the process proved to be time-consuming.



## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

The modeling approach was an iterative process wherein different model types were tried to gauge which improved classification accuracy on held-out test data and which did not. Some of the models the team attempted were nearest neighbors, ensembles, and collaborative filtering via singular value decomposition.

1. How did these decisions guide you in modeling?

These decisions guided us in modeling in that higher performing model types were thought to be used instead of any others. It was seen that SVD offered a few greater percent in accuracy than the other model types, which is fairly significant since none of the attempted other model types were able to match the baseline.

2. How do you evaluate the effectiveness of these elements?

The effectiveness of these elements was gauged in terms of running within the Colab runtime limit and evaluating predictions on held-out test data. Some of the team's earlier models had comparable accuracy to the baseline, but these were not completing within the 60 second runtime limit and so the team did not opt for these models.
Effectiveness was gauged in terms of root mean squared error, mean absolute error, and classification accuracy metrics on held-out test data.
3. What else have you tried that worked or didn't?

The model which ultimately gave the team the most success was very similar in nature to the baseline model. It was a truncated singular value decomposition performed over 6 iterations with 10 components after applying robust PCA to the unranked data examples. Some of the model types attempted which apparently decreased classification accuracy were the nearest neighbor variations and ensemble methods.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD

In [ ]:
class R_pca:  # from https://github.com/dganguli/robust-pca/blob/master/r_pca.py

    def __init__(self, D, mu=None, lmbda=None):
        self.D = D
        self.S = np.zeros(self.D.shape)
        self.Y = np.zeros(self.D.shape)

        if mu:
            self.mu = mu
        else:
            self.mu = np.prod(self.D.shape) / (4 * np.linalg.norm(self.D, ord=1))

        self.mu_inv = 1 / self.mu

        if lmbda:
            self.lmbda = lmbda
        else:
            self.lmbda = 1 / np.sqrt(np.max(self.D.shape))

    @staticmethod
    def frobenius_norm(M):
        return np.linalg.norm(M, ord='fro')

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self, tol=None, max_iter=1000, iter_print=100):
        iter = 0
        err = np.Inf
        Sk = self.S
        Yk = self.Y
        Lk = np.zeros(self.D.shape)

        if tol:
            _tol = tol
        else:
            _tol = 1E-7 * self.frobenius_norm(self.D)

        #this loop implements the principal component pursuit (PCP) algorithm
        #located in the table on page 29 of https://arxiv.org/pdf/0912.3599.pdf
        while (err > _tol) and iter < max_iter:
            Lk = self.svd_threshold(
                self.D - Sk + self.mu_inv * Yk, self.mu_inv)                            #this line implements step 3
            Sk = self.shrink(
                self.D - Lk + (self.mu_inv * Yk), self.mu_inv * self.lmbda)             #this line implements step 4
            Yk = Yk + self.mu * (self.D - Lk - Sk)                                      #this line implements step 5
            err = self.frobenius_norm(self.D - Lk - Sk)
            iter += 1
            if (iter % iter_print) == 0 or iter == 1 or iter > max_iter or err <= _tol:
                print('iteration: {0}, error: {1}'.format(iter, err))

        self.L = Lk
        self.S = Sk
        return Lk, Sk

    def plot_fit(self, size=None, tol=0.1, axis_on=True):

        n, d = self.D.shape

        if size:
            nrows, ncols = size
        else:
            sq = np.ceil(np.sqrt(n))
            nrows = int(sq)
            ncols = int(sq)

        ymin = np.nanmin(self.D)
        ymax = np.nanmax(self.D)
        print('ymin: {0}, ymax: {1}'.format(ymin, ymax))

        numplots = np.min([n, nrows * ncols])
        plt.figure()

        for n in range(numplots):
            plt.subplot(nrows, ncols, n + 1)
            plt.ylim((ymin - tol, ymax + tol))
            plt.plot(self.L[n, :] + self.S[n, :], 'r')
            plt.plot(self.L[n, :], 'b')
            if not axis_on:
                plt.axis('off')

In [ ]:
dfK = df[df.rtg!=df.rtg]          # uid-mid-rtg for Kaggle submission (contains NaN ratings)
dfX = df[df.uid.isin(dfK.uid.unique())]  # all ratings on users requiring predictions (as adjacency list)
dfX = dfX.pivot_table(index='uid', columns='mid', values='rtg')  # convert to (adjacency) matrix format
dfX_est_mean = dfX.fillna(dfX.mean())  # fill NaN with movies' average ratings, save as
n, p = dfX.shape                  # number of users and movies

random_state = 41

robust_pca_mean = R_pca(dfX_est_mean)

rpca_max_iterations = 10000
rpca_iterations_before_printing = 1000

low_rank_matrix_mean, sparse_matrix_mean = robust_pca_mean.fit(max_iter=rpca_max_iterations, iter_print=rpca_iterations_before_printing)

truncated_svd_n_components = 10
truncated_svd_n_iter = 6

df_low_rank_matrix_mean = pd.DataFrame(low_rank_matrix_mean, index=dfX.index, columns=dfX.columns)

np.random.seed(random_state)
for _ in range(truncated_svd_n_iter):                             # apply truncated SVD approximation a few times
 U_mean, D_mean, Vt_mean = svds(df_low_rank_matrix_mean.to_numpy(), k=truncated_svd_n_components)               # find k eigenvectors (U,V) and eigenvalues (D)
 df_low_rank_matrix_mean.loc[1:n, 1:p] = U_mean @ np.diag(D_mean) @ Vt_mean  # k-dimensional estimate (as np array) of dfM (we expect ratings' noise to reduce)
 mask = ~np.isnan(dfX)       # mask locations of NaN's
 df_low_rank_matrix_mean[mask] = dfX[mask]   # place original/known ratings back into estimated uid×mid matrix

# Submissions are scored on accuracy, not on continuous/regression error
dfX_est_L_mean = df_low_rank_matrix_mean.unstack().reset_index().round(0).astype(int).rename(columns={0:'rtg'})  # tall format with integer ratings

dfK_mean = dfX_est_L_mean.merge(dfK, on=['mid','uid'], suffixes=['','_']).drop('rtg_', axis=1)

dfK_mean.index = (dfK_mean.uid - 1) * dfK_mean.mid.max() + dfK_mean.mid

dfK_mean['rtg'] = dfK_mean['rtg'].clip(1, 5)  # restrict the range to 1,2,3,4,5 . Thanks to Yannis & Liz, See Piazza @67_f1

ToCSV(dfK_mean[['rtg']].sort_index(), '🍿HunterKlamut_MargaritaPrikhodko_SeiyoungCaitlynLee_Group15_Project4_Final🐍')

iteration: 1, error: 0.3533365155341691
iteration: 2, error: 0.00017538061603513284


# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

[1] Matrix Factorization-based algorithms — Surprise 1 documentation. (n.d.). Surprise.readthedocs.io. https://surprise.readthedocs.io/en/stable/matrix_factorization.html

[2] Robust Principal Component Analysis (RPCA). (n.d.). Www.youtube.com. Retrieved July 2, 2023, from https://www.youtube.com/watch?v=yDpz0PqULXQ

[3] Candès, E., Li, X., Ma, Y., & Wright, J. (2009). Robust Principal Component Analysis? https://arxiv.org/pdf/0912.3599.pdf



<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 46 sec


## 💡**Starter Ideas**

1. Tune model **hyperparameters**.
  1. SVD's $k$ might be user-perceived movie genres (perhaps, an average user categorizes movies into just 5 or 50 genres) [🎦](https://youtu.be/sooj-_bXWgk)
1. Try smarter **sampling** from the training set to avoid using all observations (due to time constraint)
  1. Some movies may have consistent high/low ratings and do not distinguish user preferences (just add to runtime)
1. Try SVD on groups (say movie genres) with the largest impact on overall accuracy
1. Try smarter initial imputation, perhaps, conditioned on movie genres or other observed features
1. Try clustering techniques from [SKL](https://scikit-learn.org/stable/modules/clustering.html)
1. Try engineering new features using Movies' data (`movies.csv`)
1. Try supervised approach on clusters or engineered features
1. Try shift/scale ratings. Recall: SVD seeks largest quadratic error from mean without shifting first, like variance does. Since $1^2\ll5^2$, SVD might work better for higher ratings.
  1. Note: the baseline model squeezes predictions towards ratings 3 and 4, which is not the original distribution.
1. Try feature restriction, selection, and regularization (see ESL pp.140-141)
1. Do a thorough EDA. Are there movies that are liked by most viewers?
1. Try Robust PCA (RPC) [🎦](https://www.youtube.com/watch?v=yDpz0PqULXQ), [RPCA Github](https://github.com/dganguli/robust-pca)
1. Find polarized movies and tune for them (like "*Napoleon Dynamite*"). See [&#127910;](https://youtu.be/ImpV70uLxyw)
1. Try KNN [📄](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.143.596&rep=rep1&type=pdf), [📄](https://scholar.google.com/scholar?q=knn+netflix+prize)
1. Learn more methods to build/use latent representations [🎦](https://youtu.be/n3RKsY2H-NE),  [📄](https://ieeexplore.ieee.org/abstract/document/5197422), [📄](https://scholar.google.com/scholar?q=netflix+prize+challenge), [&#127910;](https://www.youtube.com/results?search_query=netflix+prize+svd), [📄](https://www.aaai.org/Papers/AAAI/2002/AAAI02-029.pdf),